## setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
N_TAPS = 512
sample_rate = 24000

## load speaker response data

In [ ]:
measured_response = pd.read_csv('spkr_response.csv').drop(columns=['Impedance'])
measured_response = measured_response[(measured_response['Freq'] > 30) & (measured_response['Freq'] < sample_rate/2)]
measured_response['Phase'] = measured_response['Phase'] * np.pi/180
target_response = pd.DataFrame(np.array([[1, 0]]), columns=['Freq', 'Phase'])._append(measured_response, ignore_index=True)
target_response

phase_plot = plt.semilogx(target_response['Freq'], target_response['Phase'])
plt.xlabel('freq (Hz)')
plt.ylabel('phase (radians)')
plt.title('measured speaker response')
plt.grid()
plt.grid(which='both', axis='x')

# calculate FIR coefficients

In [ ]:
# interpolate phase measurements to linear freq intervals
k = np.linspace(0, sample_rate/2, N_taps, endpoint=False)
interpolated_phase = np.interp(k, target_response['Freq'], target_response['Phase'])

# plt.plot((freqs*sample_rate)[:50], interpolated_phase[:50])
# plt.title('interpolated phase samples')

In [ ]:
mag = np.ones(N_taps)
phase = interpolated_phase
# phase = np.random.rand(N_taps)*2*np.pi

# calculate specturum & allpass filter coeffs
spectrum = mag*np.exp(phase * 1j)
coeffs = np.fft.irfft(spectrum)

In [ ]:
# test via impulse response
impulse = np.zeros(256, dtype=np.float64)
impulse[256//2] = 1.0
result = np.convolve(impulse, coeffs, "same")
mag_result = np.absolute(np.fft.rfft(result))
arg_result = np.angle(np.fft.rfft(result))


plt.plot(mag_result, label='magnitude')
plt.plot(arg_result, label='phase')
plt.legend()
plt.title("unit sample response")

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(k, phase % (2*np.pi), label='target')
plt.plot(k, arg_result[:-1] % (2*np.pi), label='result')
plt.legend()

In [ ]:
error = ((phase % (2*np.pi)) - (arg_result[:N_taps] % (2*np.pi)))
error = np.minimum(np.abs(error), 2*np.pi - np.abs(error))
plt.plot(k, error)

## final coefficients

In [ ]:
print("coefficients:\n", coeffs)